In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import requests
import yaml

from pandas import CategoricalDtype
from bs4 import BeautifulSoup
from datetime import date
from scipy import stats

import sys
import os

config_path = r'../config/params.yml'
config = yaml.load(open(config_path), Loader=yaml.FullLoader)

data_path = config['train']['data_path']

In [ ]:
config

## Описание работы
Проект использует данные по ключевой ставке Банка России для прогнозирования ее значений в будущем с помощью моделей Prophet и NeuralProphet. Основные шаги проекта:
- Сбор исторических данных по ключевой ставке ЦБ РФ с официального сайта.
- Предобработка данных: очистка, заполнение пропусков, преобразование в формат для моделей.
- Разделение данных на обучающую и тестовую выборки.
- Обучение моделей Prophet и NeuralProphet на обучающей выборке. Prophet использует аддитивную модель с трендом и сезонностью, NeuralProphet - нейросетевую модель с аналогичными компонентами.
- Оценка качества моделей на тестовой выборке по метрикам RMSE, MAE, MAPE.
- Использование лучшей модели для прогнозирования ключевой ставки на заданный горизонт в будущем.
- Анализ полученных прогнозов, сравнение с фактическими данными и решениями ЦБ РФ по ставке
- Обучение моделей Prophet и NeuralProphet на полной выборке и предсказание ставки на будущие периоды<br>

Target - предсказание курса ключевой ставки ЦБ РФ
-   date - дата
-   key_rate - значение ключевой ставки ЦБ РФ

# Parcing data

In [ ]:
def get_dataset(config):
    """
    Парсит ключевую ставку с URL сайта ЦБ РФ и возвращает pandas DataFrame.

    Параметры:
    config (dict): Словарь конфигурации, содержащий URL для парсинга.

    Возвращает:
    pd.DataFrame: DataFrame, содержащий спарсенные данные ключевой ставки.
    """
    url = config['parcing']["URL"] + date.today().strftime('%d.%m.%Y')
    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find_all("table")

    df = pd.read_html(str(table))[0]
    df.iloc[:, 1:] /= 100
    df['Дата'] = pd.to_datetime(df['Дата'], dayfirst=True)
    df.columns = ['date', 'key_rate']

    return df

In [ ]:
# Парсинг ключевой ставки в df
df = get_dataset(config)

In [ ]:
# Общая информация о датафрейме
df.info()

In [ ]:
# Основные описательные статистики для числовых признаков
df.iloc[:, 1:].describe()

In [ ]:
# Смотрим график курса ключевой ствки ЦБ РФ и график распределения
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

sns.set_theme(style="whitegrid", palette="Accent")

sns.lineplot(x='date', y='key_rate', data=df, label='Ставка', ax=ax[0])
ax[0].set_xlabel('График ставки рефинансирования РФ')
ax[0].set_ylabel('Значение ставки')
ax[0].legend(loc='best')
ax[0].grid(True)

sns.kdeplot(x=df['key_rate'], ax=ax[1], fill=True)
ax[1].grid(True)
ax[1].set_xlabel('График распределения ставки')
ax[1].set_ylabel('Плотность вероятности')
plt.show()

# EDA

In [ ]:
# Определение категориальных типов данных для дней недели и месяцев
# cat_day = CategoricalDtype(categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered= True)
# cat_month = CategoricalDtype(categories=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], ordered= True)

def create_features(data, col_datetime):
    """
    Создание новых признаков из столбца datetime в pandas DataFrame.

    Параметры:
    data (pd.DataFrame): Входной DataFrame
    col_datetime (str): Имя столбца datetime

    Возвращает:
    pd.DataFrame: DataFrame с добавленными новыми признаками
    """
    # Создание копии df
    data = data.copy()

    # Определение категориальных типов данных для дней недели и месяцев
    cat_day = CategoricalDtype(categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered= True)
    cat_month = CategoricalDtype(categories=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], ordered= True)

    # Преобразование столбца datetime в формат datetime
    data[col_datetime] = pd.to_datetime(data[col_datetime])

    # Создание новых признаков
    data['weekday'] = data[col_datetime].dt.day_name().astype(cat_day)  # День недели
    data['month'] = data[col_datetime].dt.month_name().astype(cat_month)  # Месяц года
    data['year'] = data[col_datetime].dt.year  # Год
    data['quarter'] = data[col_datetime].dt.quarter  # Квартал года
    data['date_offset'] = (data[col_datetime].dt.month * 100 + data[col_datetime].dt.day - 320) % 1300  # Пользовательский признак смещения даты
    data['season'] = data[col_datetime].dt.month.map({1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring', 5: 'Spring', 6: 'Summer', 7: 'Summer', 8: 'Summer', 9: 'Autumn', 10: 'Autumn', 11: 'Autumn', 12: 'Winter'})  # Сезон года

    return data

In [ ]:
# Создание признаков
df_features = create_features(data=df, col_datetime='date')

In [ ]:
def plot_features(df_features):
    """
    Создает два графика для анализа ключевой ставки:
    1. Бары максимальных значений ставки по годам.
    2. Боксплот распределения ставки по дням недели и сезонам.

    Параметры:
    df_features (pd.DataFrame): DataFrame, содержащий данные о ключевой ставке, днях недели, сезоны.

    Возвращает:
    Два графика
    """
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))

    year_group = pd.DataFrame(df_features.groupby('year')['key_rate'].max()).reset_index().sort_values('key_rate')

    sns.barplot(data=year_group, x='year', y='key_rate', ax=ax[0])
    ax[0].set_xlabel('Год')
    ax[0].set_ylabel('Значение ставки')

    sns.boxplot(data=df_features, x='weekday', y='key_rate', hue='season', ax=ax[1], linewidth=2)
    ax[1].set_xlabel('День недели')
    ax[1].set_ylabel('Значение ставки')

    plt.show()

In [ ]:
plot_features(df_features)

## Дропаем редкие субботние ставки

In [ ]:
# Дропаем редкие субботние ключевые ставки
drop_trash = df_features[(df_features.weekday == 'Saturday')].index
df = df[~df.index.isin(drop_trash)]
df = df.reset_index(drop=True)

Смотрим график после дропа суббот

In [ ]:
# Создаем df с признаками после дропа суббот
df_features = create_features(data=df, col_datetime='date')

In [ ]:
# Смотрим графики для анализа после дропа суббот
plot_features(df_features)

## Поиск и удаление статистических выбросов

## Определение выбросов при помощи IQR

In [ ]:
# Определение последней даты, для заполнения графика интерполяцией
last_date = df['date'].max()

# Настройка фильтрации данных до последней даты
mask = df['date'] < last_date

# Вычисление межквартильного размаха (IQR)
q1, q3 = df.loc[mask, 'key_rate'].quantile([0.25, 0.75])
iqr = q3 - q1

# Определение границ для выбросов
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Замена выбросов на NaN для подсчёта выбросов
df_filtered = df.copy()
df_filtered.loc[(df_filtered['key_rate'] < lower_bound) & mask, 'key_rate'] = np.nan
df_filtered.loc[(df_filtered['key_rate'] > upper_bound) & mask, 'key_rate'] = np.nan

# Считаем количество статистических выбросов
print(df_filtered['key_rate'].isna().sum())

In [ ]:
def plot_interpolate(df, df_filtered):
    """
    Визуализация оригинальных и отфильтрованных данных ключевой ставки.

    Параметры:
    df (pandas.DataFrame): Оригинальные данные ключевой ставки.
    df_filtered (pandas.DataFrame): Отфильтрованные данные ключевой ставки.
    """
    fig, ax = plt.subplots(figsize=(15, 5))
    sns.set_theme(style="whitegrid", palette="Accent")

    sns.lineplot(x='date', y='key_rate', data=df, label='Оригинальные данные', ax=ax)
    sns.lineplot(x='date', y='key_rate', data=df_filtered, label='Отфильтрованные данные', ax=ax)

    ax.set_xlabel('Год')
    ax.set_ylabel('Значение ставки')
    ax.set_title('Визуализация после вычисления выбросов')
    ax.legend(loc='best')
    ax.grid(True)

    plt.show()

In [ ]:
# Визуализация после вычисления и удаления выбросов
plot_interpolate(df, df_filtered)

In [ ]:
# Интерполируем пропущенные значения
df_filtered['key_rate'] = df_filtered['key_rate'].interpolate(method='nearest', order=3)

# Считаем количество статистических выбросов после интерполяции
print(df_filtered['key_rate'].isna().sum())

In [ ]:
# Создаем df с признаками после замены статистических выбросов интерполяцией
df_features = create_features(data=df_filtered, col_datetime='date')

In [ ]:
# Смотрим графики после замены статистических выбросов интерполяцией
plot_features(df_features)

In [ ]:
# Смотрим график после замены выбросов интерполяцией пропущенных значений
plot_interpolate(df, df_filtered)

In [ ]:
# сохранение отфилтрованных данных в df
df = df_filtered

# Перенаименование названия колонок для prophet
df.columns = ['ds', 'y']

# Период, который надо отрезать и предсказать (проверка модели)
pred_days = int(df.shape[0]*config['parcing']['pred_days'])


In [ ]:
# Сортируем данные по возрастанию для корректного отображения разделения графика
df = df.sort_values('ds')
df = df.reset_index(drop=True)

In [ ]:
# Разделение данных на train, test
df_train = df[:-pred_days]
df_test = df[-pred_days:]


In [ ]:
# Отображаем график с разделением train, test для наглядности
fig, ax = plt.subplots(figsize = (10, 5))

fig.set_figheight(5)
fig.set_figwidth(15)
df_train.set_index('ds').plot(ax=ax, label= 'train', title='Визуальное разделение на тестовые и тренировочные данные')
df_test.set_index('ds').plot(ax=ax, label='test')
ax.axvline(df_train['ds'][-1:].values, ls='--', color='black')
ax.legend(['df_train', 'df_test'])
ax.set_ylabel('Значение ставки')
ax.set_xlabel('Год')
plt.show()

# Сохранение датасета

In [ ]:
# Сохранение DataFrame df в файл data/df.csv
df.to_csv(f'{data_path}/df.csv', index=False)
#df.to_csv('../data/df.csv', index=False)

# Сохранение DataFrame df_train в файл data/df_train.csv
df_train.to_csv(f'{data_path}/df_train.csv', index=False)

# Сохранение DataFrame df_test в файл data/df_test.csv
df_test.to_csv(f'{data_path}/df_test.csv', index=False)
